<a href="https://colab.research.google.com/github/osgeokr/GEE-PAM-Book/blob/main/%EC%8B%9D%EC%83%9D%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import ee
import geemap
import pandas as pd
import geopandas as gpd
import json
import time

# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='ee-foss4g')

In [3]:
with open('SEORAKSAN.geojson') as f:
    geojson = json.load(f)
seoraksan = ee.FeatureCollection(geojson)

In [4]:
from ipyleaflet import TileLayer

# Vworld 배경지도 객체
vworld_base = TileLayer(
    url='https://xdworld.vworld.kr/2d/Base/service/{z}/{x}/{y}.png',
    name='Vworld Base',
    attribution='Vworld',
)

# 설악산국립공원 경계 가시화
m = geemap.Map(width="800px", height="500px")
m.add_layer(vworld_base)
m.addLayer(seoraksan, {'color': 'green'}, 'Seoraksan') # 레이어 추가
m.centerObject(seoraksan, 11) # 지도의 중심 설정
m # 지도 객체 출력

Map(center=[38.136218061277916, 128.41248414920196], controls=(WidgetControl(options=['position', 'transparent…

In [18]:
# 시간 범위 설정
start_date = '2024-01-01'
end_date = '2024-02-29'

# Landsat 9 컬렉션 불러오기
landsat9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
    .filterDate(start_date, end_date) \
    .filterBounds(seoraksan)

# 구름 마스크 함수 정의
def maskL9sr(image):
    # 품질 밴드 가져오기
    qa = image.select('QA_PIXEL')
    # 구름 마스크 정의
    cloud = qa.bitwiseAnd(1 << 3).eq(0)
    shadow = qa.bitwiseAnd(1 << 4).eq(0)
    mask = cloud.And(shadow)
    return image.updateMask(mask)

# Applies scaling factors
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(
        thermal_bands, None, True
    )

# 구름 제거 적용 및 스케일링 팩터 적용
processed_landsat9 = landsat9.map(apply_scale_factors)

# 중간값 이미지 계산
median_image = processed_landsat9.median()

# 시각화 파라미터 설정
visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

# 결과 시각화
Map = geemap.Map()
Map.centerObject(seoraksan, 10)
Map.addLayer(median_image, visualization, 'Median Image')
Map.addLayer(seoraksan, {}, 'Seoraksan')
Map

Map(center=[38.136218061277916, 128.41248414920196], controls=(WidgetControl(options=['position', 'transparent…

In [16]:
# 시간 범위 설정
start_date = '2024-01-01'
end_date = '2024-02-29'

# Landsat 9 컬렉션 불러오기
landsat9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
    .filterDate(start_date, end_date) \
    .filterBounds(seoraksan)

# 구름 마스크 함수 정의
def maskL9sr(image):
    # 품질 밴드 가져오기
    qa = image.select('QA_PIXEL')
    # 구름 마스크 정의
    cloud = qa.bitwiseAnd(1 << 3).eq(0)
    shadow = qa.bitwiseAnd(1 << 4).eq(0)
    mask = cloud.And(shadow)
    return image.updateMask(mask)

# 구름 제거 적용
cloud_free_landsat9 = landsat9.map(maskL9sr)

# 중간값 이미지 계산
median_image = cloud_free_landsat9.median()

# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

AttributeError: 'Image' object has no attribute 'map'

In [15]:
# 결과 시각화
m = geemap.Map(width="800px", height="500px")
m.addLayer(median_image, {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 1}, 'Median Image')
m.addLayer(seoraksan, {'color': 'green'}, 'Seoraksan') # 레이어 추가
m.centerObject(seoraksan, 11) # 지도의 중심 설정
m # 지도 객체 출력

Map(center=[38.136218061277916, 128.41248414920196], controls=(WidgetControl(options=['position', 'transparent…

In [19]:
def mask_s2_clouds(image):
    # QA(Quality Assurance) 밴드 사용, S2에서 구름 마스킹
    qa = image.select('QA60')

    # 비트 10은 구름(clouds), 11은 성층운(cirrus)
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # 구름과 성층운이 0이면 맑은 상태로 간주함.
    mask = (
        qa.bitwiseAnd(cloud_bit_mask)
        .eq(0)
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    )

    return image.updateMask(mask).divide(10000) # 스케일링

In [20]:
# Sentinel-2 이미지 선택 및 필터링
s2_images = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterDate("2024-01-01", "2024-01-31")
    .filterBounds(seoraksan)
    # 구름이 5% 미만인 이미지 필터링
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 5))
    .map(mask_s2_clouds)
)

# 이미지 컬렉션의 이미지 개수 확인
image_count = s2_images.size()

# 이미지 개수 출력
print("Image count:", image_count.getInfo())

Image count: 4


In [21]:
# 중간값 이미지 계산
s2_image = s2_images.median()

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map(width="800px", height="500px")
m.add_layer(s2_image, visualization, 'RGB')
m.centerObject(seoraksan, 11) # 지도의 중심 설정
m # 지도 객체 출력

Map(center=[38.136218061277916, 128.41248414920196], controls=(WidgetControl(options=['position', 'transparent…

In [22]:
# 밴드 이름 목록
band_names = s2_image.bandNames()
print(band_names.getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60']


In [31]:
# B2, B3, B4, B8 밴드 선택
rgb_image = s2_image.select(['B2', 'B3', 'B4'])

In [32]:
# 이미지를 Google Drive에 내보내기
task = ee.batch.Export.image.toDrive(
    image=rgb_image,
    description='S2_RGB_202401',
    folder='export',
    scale=10,  # 이미지의 해상도
    region=seoraksan.geometry(),  # 내보낼 영역
    maxPixels=1e13,
    fileFormat='GeoTIFF'
)

# 내보내기 작업 시작
task.start()

# 내보내기 작업 상태 확인
print('Export task started. Checking status...')

while True:
    status = task.status()
    state = status['state']
    print('Polling for task (id: {}). Status: {}'.format(task.id, state))
    if state in ['COMPLETED', 'FAILED']:
        break
    time.sleep(30)  # 30초 간격으로 상태 확인

# 완료 후 상태 출력
print('Task completed. Final status:')
print(status)

Export task started. Checking status...
Polling for task (id: EFZJFH4Y3CFY7HOB4UACTCTX). Status: READY
Polling for task (id: EFZJFH4Y3CFY7HOB4UACTCTX). Status: RUNNING
Polling for task (id: EFZJFH4Y3CFY7HOB4UACTCTX). Status: RUNNING
Polling for task (id: EFZJFH4Y3CFY7HOB4UACTCTX). Status: RUNNING
Polling for task (id: EFZJFH4Y3CFY7HOB4UACTCTX). Status: RUNNING
Polling for task (id: EFZJFH4Y3CFY7HOB4UACTCTX). Status: RUNNING
Polling for task (id: EFZJFH4Y3CFY7HOB4UACTCTX). Status: RUNNING
Polling for task (id: EFZJFH4Y3CFY7HOB4UACTCTX). Status: RUNNING
Polling for task (id: EFZJFH4Y3CFY7HOB4UACTCTX). Status: RUNNING
Polling for task (id: EFZJFH4Y3CFY7HOB4UACTCTX). Status: RUNNING
Polling for task (id: EFZJFH4Y3CFY7HOB4UACTCTX). Status: RUNNING
Polling for task (id: EFZJFH4Y3CFY7HOB4UACTCTX). Status: RUNNING
Polling for task (id: EFZJFH4Y3CFY7HOB4UACTCTX). Status: RUNNING
Polling for task (id: EFZJFH4Y3CFY7HOB4UACTCTX). Status: RUNNING
Polling for task (id: EFZJFH4Y3CFY7HOB4UACTCTX). Sta

In [ ]:
# NDVI 계산 (NIR = B8, Red = B4)
ndvi_image = s2_image.normalizedDifference(['B8', 'B4']).rename('NDVI').clip(seoraksan.geometry())

In [34]:
# NDSI 계산 (NIR = B8, Red = B4)
ndsi_image = s2_image.normalizedDifference(['B3', 'B11']).rename('NDSI')

In [35]:
# 이미지를 Google Drive에 내보내기
task = ee.batch.Export.image.toDrive(
    image=ndsi_image,
    description='S2_NDSI_202401',
    folder='export',
    scale=10,  # 이미지의 해상도
    region=seoraksan.geometry(),  # 내보낼 영역
    maxPixels=1e13,
    fileFormat='GeoTIFF'
)

# 내보내기 작업 시작
task.start()

# 내보내기 작업 상태 확인
print('Export task started. Checking status...')

while True:
    status = task.status()
    state = status['state']
    print('Polling for task (id: {}). Status: {}'.format(task.id, state))
    if state in ['COMPLETED', 'FAILED']:
        break
    time.sleep(30)  # 30초 간격으로 상태 확인

# 완료 후 상태 출력
print('Task completed. Final status:')
print(status)

Export task started. Checking status...
Polling for task (id: ZRQRXOSXISY2AAXYGVGXPF2I). Status: READY
Polling for task (id: ZRQRXOSXISY2AAXYGVGXPF2I). Status: RUNNING
Polling for task (id: ZRQRXOSXISY2AAXYGVGXPF2I). Status: RUNNING
Polling for task (id: ZRQRXOSXISY2AAXYGVGXPF2I). Status: RUNNING
Polling for task (id: ZRQRXOSXISY2AAXYGVGXPF2I). Status: RUNNING
Polling for task (id: ZRQRXOSXISY2AAXYGVGXPF2I). Status: RUNNING
Polling for task (id: ZRQRXOSXISY2AAXYGVGXPF2I). Status: RUNNING
Polling for task (id: ZRQRXOSXISY2AAXYGVGXPF2I). Status: RUNNING
Polling for task (id: ZRQRXOSXISY2AAXYGVGXPF2I). Status: RUNNING
Polling for task (id: ZRQRXOSXISY2AAXYGVGXPF2I). Status: RUNNING
Polling for task (id: ZRQRXOSXISY2AAXYGVGXPF2I). Status: RUNNING
Polling for task (id: ZRQRXOSXISY2AAXYGVGXPF2I). Status: RUNNING
Polling for task (id: ZRQRXOSXISY2AAXYGVGXPF2I). Status: RUNNING
Polling for task (id: ZRQRXOSXISY2AAXYGVGXPF2I). Status: RUNNING
Polling for task (id: ZRQRXOSXISY2AAXYGVGXPF2I). Sta

In [ ]:
# NDVI 이미지의 값의 범위를 계산
ndvi_range = ndvi_image.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=ndvi_image.geometry(),
    scale=10,
    maxPixels=1e13
)

# 결과 출력
ndvi_range_info = ndvi_range.getInfo()
min_ndvi = ndvi_range_info['NDVI_min']
max_ndvi = ndvi_range_info['NDVI_max']
print('NDVI 값의 범위:')
print('최소값:', min_ndvi)
print('최대값:', max_ndvi)

NDVI 값의 범위:
최소값: -0.3523421769903362
최대값: 1


In [ ]:
# 전체 이미지의 픽셀 면적 계산
area_image = ee.Image.pixelArea()
total_area = area_image.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=ndvi_image.geometry(),  # 이미지의 전체 영역
    scale=10,
    maxPixels=1e13
)

# 결과 면적 출력
total_area_sqm = total_area.getInfo()['area']  # 면적 값 가져오기
total_area_sqkm = total_area_sqm / 1e6  # 제곱킬로미터로 변환
print('NDVI 이미지의 전체 면적:', total_area_sqkm, 'square kilometers')

NDVI 이미지의 전체 면적: 398.0729173340117 square kilometers


In [ ]:
# NDVI 값을 기준으로 마스크 적용 (-1에서 1 사이의 모든 값 포함)
# 기존 마스크를 제거하고 NDVI 값의 범위로만 마스크 설정
ndvi_image_unmasked = ndvi_image.unmask()
ndvi_mask = ndvi_image_unmasked.gte(-1).And(ndvi_image_unmasked.lte(1))
masked_ndvi = ndvi_image_unmasked.updateMask(ndvi_mask)

# 이미지 영역 내에서 마스크된 이미지의 픽셀 수 계산
area_image = masked_ndvi.multiply(ee.Image.pixelArea())
masked_area = area_image.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=ndvi_image.geometry(),
    scale=10,
    maxPixels=1e13
)

# 결과 면적 출력
masked_area_sqm = masked_area.getInfo()['NDVI']  # NDVI 밴드의 면적 값 가져오기
masked_area_sqkm = masked_area_sqm / 1e6  # 제곱킬로미터로 변환
print('NDVI 값이 -1에서 1 사이인 전체 면적:', masked_area_sqkm, 'square kilometers')

NDVI 값이 -1에서 1 사이인 전체 면적: 93.8246108424691 square kilometers
